# Chapter 2 - Linear systems of equations

https://tobydriscoll.net/fnc-julia/linsys/matrices.html

In [ ]:
using Revise;
using Test;
using MyFNC;
using FundamentalsNumericalComputation;
# import Pkg; Pkg.add("AppleAccelerate")
using AppleAccelerate;

# 2.2 Computing with matrices

In [ ]:
# Either semicolon or newline creates a row
A = [ 1 2 3 4 5; 50 40 30 20 10 
π √2 exp(1) (1+√5)/2 log(3)];

display(A)

In [ ]:
m,n = size(A)

## Matrix/vector basics

In [ ]:
# Vector only has one dimension
x = [ 3, 3, 0, 1, 0 ]
@show size(x)
@show typeof(x)
@show size(x')  # adjoint of vector is row matrix.
@show size(x'') # adjoin of row matrix is a vector

# Can use comma or newline
x′ = [
    1
    2
    3
    4
    5
] 
@show size(x′)
@show typeof(x′)

# Without comma it becomes a row matrix
X = [ 3 3 1im 1 0]
@show size(X)
@show typeof(X)

# Can take adjoint / complex conjugate with single quote
X'

#### Blocks

You can concate matrix and vectors in math notation, as long as the dimensions are compatible.

In [ ]:
[x x]

In [ ]:
[x
 x]

In [ ]:
# incompatible
[x x; x]

### Building matrices

With ranges

In [ ]:
display([1:3 1:3])

In [ ]:
display([(1:3)'; (1:3)'])

In [ ]:
# Final semicolon is important, and converts to a matrix
display([ [(1:3)'; (1:3)'; (1:3)']; ])
display([ [(1:3)'; (1:3)'; (1:3)'] ])

In [ ]:
# Putting it all together
display([ [(1:3)'; (1:3)'; (1:3)'] 4:6])


In [ ]:
### Non-integer ranges

In [ ]:
@show collect(1:10:0.1) # no worky!
@show collect(1:10:(0.1)) # no worky!
@show collect(1:0.1:2) # distance is the middle parameter!

In [ ]:
# range computes a range with the given number of numbers
@show range(-1.5, 1.5, 5)
@show size(range(-1.5, 1.5, 5))
@show collect(range(-1.5, 1.5, 5))

In [ ]:
s = range(-1, 1, length=5) # kwarg for readability
# s[0] # cannot access zero index
# s[-1] # cannot access negative indices

# end is magic and is the last index in the range.
@show s[end]  


# ranges are lazy
@show range(-1, 1, length=typemax(Int))[end-1]

In [ ]:
# RIP memory
# collect(range(-1, 1, length=typemax(Int32)))
# no way to clear used memory in REPL? Restart kernel to free this mistake.

In [ ]:
using Profile
Profile.clear()

### Array indexing with ranges
Easily produce some intricate patterns

In [ ]:
A = zeros(5,6)

In [ ]:
A = zeros(5,6);

A[1:2:end,2:2:end] .= 1
display(A)

A[2:2:end,1:2:end] .= 2
display(A)

In [ ]:
using Plots;
Plots.heatmap(A)

### Indexing arrays with tuples, and spreading

In [ ]:
A = [1 2 ; 3 4]
display(A)

A[2,2] = 9
display(A)

typeof((2,2))
bottomright = (2,2)
topleft = (1,1)

# Doesn't work, cannot index with tuple
# A[bottomright] = 99

# Works! Can spread a tuple.
A[bottomright...] = 99

# Doesn't work, setindex expects a list of indices, not a tuple
# setindex!(A, bottomright, 99) 

# Works! Can spread the tuple, and setindex! expects second arg to be the value.
setindex!(A, 88, topleft...) 
display(A)


### 2.2 Exercises

In [ ]:
A = [2 1 1 0
     0 -1 4 1
     2 2 0 -2
     1 3 -1 5]

B = [3 -1 0 2
     7 1 0 2];

u = [2, -1, 3, 1]

v = [pi ℯ]
@show v

In [ ]:
# a)
# DimensionMismatch: matrix A has dimensions (4,4), matrix B has dimensions (2,4)
# A*B

# b)
@show size(B), size(A)
display(B*A)

# ∘ = circ

using LinearAlgebra

C_11 = B[1,:] ⋅ A[:,1] 
C_12 = B[1,:] ⋅ A[:,2] 
C_13 = B[1,:] ⋅ A[:,3] 
C_14 = B[1,:] ⋅ A[:,4] 

C_21 = B[2,:] ⋅ A[:,1] 
C_22 = B[2,:] ⋅ A[:,2] 
C_23 = B[2,:] ⋅ A[:,3] 
C_24 = B[2,:] ⋅ A[:,4] 

C = [C_11 C_12 C_13 C_14
     C_21 C_22 C_23 C_24]

In [ ]:
# c)
# DimensionMismatch: matrix A has dimensions (2,1), matrix B has dimensions (2,4)
# v' * B
# For this to work, use v instead.
v * B


In [ ]:
# d)
B * u

In [ ]:
# e)
[u A*u A^2*u A^3*u]

### Exercise 2.2.3

In [ ]:
u = [1, 3, 5, 7, 9, 11];
v = [-60, -50, -40, -30, -20, -10];

In [ ]:
u' * v

In [ ]:
v' * u

In [ ]:
u * v'

In [ ]:
v * u'

### Exercise 2.2.4

In [ ]:
A = rand(3,4)
B = rand(4,2)

display((A*B)')

display((B' * A'))

display(norm((A*B)' - (B' * A')))

## 2.3 Linear systems

Back substitution algorithm for solver upper diagonal systems.


In [ ]:
"""
    backsub(U,b)

Solve the upper triangular linear system with matrix `U` and
right-hand side vector `b`. Ux=b
"""
function backsub(U,b)
    n = length(b)
    x = zeros(Float64, n)
    x[n] = b[n]/U[n,n]
    for i = n-1:-1:1
        s = sum(U[i,i+1:n] .* x[i+1:n])
        x[i] = (b[i] - s)/U[i,i]
    end
    return x
end

A = [1 2 3; 0 4 5; 0 0 6]
b = [1, 2, 3]
@show A \ b;
@show backsub(A, b);

In [ ]:
# making a random matrix
A = rand(1:9, 5,5)

# taking the upper triangular part
U = triu(A)

# taking the lower triangular part
L = tril(A)

In [ ]:
α = 0.3;
β = 1e12;
U = diagm(0=>ones(5), 1=>[-1,-1,-1,-1])
display(U)
U[1,4] = α-β
U[1,5] = β
b = [α, 0, 0, 0, 1]

x_exact = ones(5)
@show cond(U)
x = backsub(U,b)
@show x - x_exact
@show U\b-x_exact


### Exercise 2.3.1

In [ ]:
# Not solveable, has infinite solutions
# A = [0 1; 0 0]
# b = [1, 1]
# A \ b


### Exercise 2.3.2
Solve the triangular systems by hand.

### Exercise 2.3.3

In [ ]:
using MyFNC

# (a) Part 1
L = [-2 0 0; 1 -1 0; 3 2 1]
b = [-4, 2, 1]
@show x = MyFNC.forwardsub(L, b)

# Should be equal
@test x == FNC.forwardsub(L, b)
# Should be zero
@test L*x - b ≈ zeros(3)

In [ ]:

# (a)
L = [-2 0 0; 1 -1 0; 3 2 1]
b = [-4, 2, 1]
@show x = MyFNC.forwardsub(L, b)
@test x == FNC.forwardsub(L, b)
@test L*x - b ≈ zeros(3)

# (b)
L = [4 0 0 0; 1 -2 0 0; -1 4 4 0; 2 -5 5 1]
b = [-4, 1, -3, 5]
@show x = MyFNC.forwardsub(L, b)
@test x == FNC.forwardsub(L, b)
@test L*x - b ≈ zeros(4)

# (c)
U = [3 2 1; 0 1 -1; 0 0 2]
b = [1, 2, -4]
@show x = MyFNC.backsub(U, b)
@test x == FNC.backsub(U, b)
@test U*x - b ≈ zeros(3)

### Exercise 2.3.4

In [ ]:
# (a)
U = [3 1 0; 0 -1 -2; 0 0 3]
b = [1, 1, 6]
@show x = MyFNC.backsub(U, b)
@test U*x - b ≈ zeros(3)

# (b)
U = [3 1 0 6; 0 -1 -2 7; 0 0 3 4; 0 0 0 5]
b = [4, 1, 1, 5]
@show x = MyFNC.backsub(U, b)
@test U*x - b ≈ zeros(4)

### Exercise 2.3.5

In [ ]:
function solve235b(n)
    τ = 10 # N
    g = -9.81 # m/s²
    m = ones(n-1) / (10n)
    A = diagm(0=>-2*ones(n-1), 1=>ones(n-2), -1=>ones(n-2))
    f = -g/(τ*n) *  m
    q = A\f
    return [0; q; 0]
end

p = plot(range(0, 1, 40+1), solve235b(40), label="n=40")
p = plot!(range(0, 1, 8+1), solve235b(8), label="n=8")
title!(L"m_k = 1/(10n)")
display(p)

function solve235c(n)
    τ = 10 # N
    g = -9.81 # m/s²
    m = (1:n-1) / (5n^2)
    A = diagm(0=>-2*ones(n-1), 1=>ones(n-2), -1=>ones(n-2))
    f = -g/(τ*n) *  m
    q = A\f
    return [0; q; 0]
end

p = plot(range(0, 1, 40+1), solve235c(40), label="n=40")
p = plot!(range(0, 1, 8+1), solve235c(8), label="n=8")
title!(L"m_k = k/(5n^2)")
display(p)



## 2.5 Efficiency of matrix computations

In [ ]:
# Multiplication
A = rand(1000,1000)
B = tril(A)
C = LowerTriangular(B)

A*A;
tA = @elapsed for i in 1:100; A*A; end
@show tA

B*B;
tB = @elapsed for i in 1:100; B*B; end
@show tB

C*C;
tC = @elapsed for i in 1:100; C*C; end
@show tC

In [ ]:
# Solving
b = rand(1000)
A = rand(1000,1000)
B = tril(A)
C = LowerTriangular(B)

# Quite slow
A\b;
tA = @elapsed for i in 1:100; A\b; end
@show tA


# Quite fast
B\b;
tB = @elapsed for i in 1:100; B\b; end
@show tB

# Triangular systems are drastically faster to solve
C\b;
tC = @elapsed for i in 1:100; C\b; end
@show tC

## 2.6 Row pivoting

In [ ]:
A₁ = [2 0 4 3 ; -4 5 -7 -10 ; 1 15 2 -4.5 ; -2 0 2 -13];
@show cond(A₁)

L,U=factorize(A₁)
display(L)
display(U)

L,U = lu(A₁)
@show cond(A₁)
@show cond(L*U)
@show A₁-L*U

In [ ]:
A₁ = [2 0 4 3 ; -4 5 -7 -10 ; 1 15 2 -4.5 ; -2 0 2 -13];
A₁[[2,4],:] = A₁[[4,2],:]
@show cond(A₁)

@show L,U=factorize(A₁)
display(L)
display(U)

L,U = lu(A₁)
@show cond(A₁)
@show cond(L*U)
@show A₁-L*U

### QR factorization is smart

In [ ]:
# Make a random symmetric positive definite matrix
A = rand(10,8)
N = A' * A + 10*I

qr(N)